# 효율적인 정리를 위한 class화

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
    
    def predict(self, x_test, keep_prop=1.0):
        return self.sess.run(self.logits, feed_dict = {self.X: x_test, self.keep_prob: keep_prop})
    
    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})
    
    def train(self, x_data, y_data, keep_prop=0.7):
        return self.sess.run([cost, optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

In [3]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [4]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [5]:
# Convolution layer1
conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding="SAME", strides= 2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)
# Convolution layer2
conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], padding="SAME", strides= 2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7, training=True)
# Convolution layer3
conv3 = tf.layers.conv2d(inputs=dropout2, filters=64, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], padding="SAME", strides= 2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7, training=True)
#FC layer
flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
dropout4 = tf.layers.dropout(inputs=dense4, rate = 0.5, training=True)

C:\Users\since\anaconda3\lib\site-packages\keras\legacy_tf_layers\convolutional.py:536: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C:\Users\since\anaconda3\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\since\anaconda3\lib\site-packages\keras\legacy_tf_layers\pooling.py:554: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
C:\Users\since\anaconda3\lib\site-packages\keras\legacy_tf_layers\core.py:393: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use 

In [6]:
W3 = tf.get_variable("W3", shape=[128 * 4 * 4, 10], initializer= tf.compat.v1.keras.initializers.glorot_normal)
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(flat, W3) + b

In [19]:
learning_rate = 0.1
training_epochs =5
batch_size = 100
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [20]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print("Learning Strarted!")

Learning Strarted!


In [15]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

  0%|          | 0/600 [00:00<?, ?it/s]


AttributeError: 'Model' object has no attribute 'keep_prob'

In [ ]:
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c / total_batch
    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))
print("Learning Finished!")

Learning started. It takes sometime.


  0%|          | 0/600 [00:00<?, ?it/s]